## ਐਮਬੈਡਿੰਗਜ਼

ਸਾਡੇ ਪਿਛਲੇ ਉਦਾਹਰਨ ਵਿੱਚ, ਅਸੀਂ `vocab_size` ਦੀ ਲੰਬਾਈ ਵਾਲੇ ਉੱਚ-ਡਾਈਮੇਂਸ਼ਨਲ ਬੈਗ-ਆਫ-ਵਰਡਜ਼ ਵੇਕਟਰਾਂ 'ਤੇ ਕੰਮ ਕੀਤਾ ਸੀ, ਅਤੇ ਅਸੀਂ ਘੱਟ-ਡਾਈਮੇਂਸ਼ਨਲ ਪੋਜ਼ੀਸ਼ਨਲ ਪ੍ਰਦਰਸ਼ਨ ਵੇਕਟਰਾਂ ਨੂੰ ਸਪਸ਼ਟ ਤੌਰ 'ਤੇ sparse ਇੱਕ-ਹਾਟ ਪ੍ਰਦਰਸ਼ਨ ਵਿੱਚ ਬਦਲਿਆ ਸੀ। ਇਹ ਇੱਕ-ਹਾਟ ਪ੍ਰਦਰਸ਼ਨ ਮੈਮੋਰੀ-ਕੁਸ਼ਲ ਨਹੀਂ ਹੈ। ਇਸ ਤੋਂ ਇਲਾਵਾ, ਹਰ ਸ਼ਬਦ ਨੂੰ ਇੱਕ-ਦੂਜੇ ਤੋਂ ਅਜ਼ਾਦ ਤੌਰ 'ਤੇ ਟ੍ਰੀਟ ਕੀਤਾ ਜਾਂਦਾ ਹੈ, ਇਸ ਲਈ ਇੱਕ-ਹਾਟ ਕੋਡ ਕੀਤੇ ਵੇਕਟਰ ਸ਼ਬਦਾਂ ਦੇ ਅਰਥਾਤਮਕ ਸਮਾਨਤਾਵਾਂ ਨੂੰ ਪ੍ਰਗਟ ਨਹੀਂ ਕਰਦੇ।

ਇਸ ਯੂਨਿਟ ਵਿੱਚ, ਅਸੀਂ **News AG** ਡੇਟਾਸੈਟ ਦੀ ਖੋਜ ਜਾਰੀ ਰੱਖਾਂਗੇ। ਸ਼ੁਰੂ ਕਰਨ ਲਈ, ਆਓ ਡੇਟਾ ਲੋਡ ਕਰੀਏ ਅਤੇ ਪਿਛਲੇ ਯੂਨਿਟ ਤੋਂ ਕੁਝ ਪਰਿਭਾਸ਼ਾਵਾਂ ਪ੍ਰਾਪਤ ਕਰੀਏ।


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### ਐਮਬੈਡਿੰਗ ਕੀ ਹੈ?

**ਐਮਬੈਡਿੰਗ** ਦਾ ਵਿਚਾਰ ਇਹ ਹੈ ਕਿ ਸ਼ਬਦਾਂ ਨੂੰ ਘੱਟ-ਮਾਪ ਦੇ ਸੰਘਣੇ ਵੇਕਟਰਾਂ ਵਿੱਚ ਦਰਸਾਇਆ ਜਾਵੇ ਜੋ ਸ਼ਬਦ ਦੇ ਅਰਥ ਨੂੰ ਦਰਸਾਉਂਦੇ ਹਨ। ਅਸੀਂ ਬਾਅਦ ਵਿੱਚ ਚਰਚਾ ਕਰਾਂਗੇ ਕਿ ਅਰਥਪੂਰਨ ਸ਼ਬਦ ਐਮਬੈਡਿੰਗ ਕਿਵੇਂ ਬਣਾਈ ਜਾ ਸਕਦੀ ਹੈ, ਪਰ ਇਸ ਸਮੇਂ ਲਈ, ਐਮਬੈਡਿੰਗ ਨੂੰ ਇੱਕ ਤਰੀਕੇ ਵਜੋਂ ਸੋਚੋ ਜੋ ਸ਼ਬਦ ਵੇਕਟਰ ਦੀ ਮਾਪ ਨੂੰ ਘਟਾਉਂਦਾ ਹੈ।

ਇਸ ਤਰ੍ਹਾਂ, ਇੱਕ ਐਮਬੈਡਿੰਗ ਲੇਅਰ ਇੱਕ ਸ਼ਬਦ ਨੂੰ ਇਨਪੁਟ ਵਜੋਂ ਲੈਂਦੀ ਹੈ ਅਤੇ ਨਿਰਧਾਰਤ `embedding_size` ਦਾ ਆਉਟਪੁੱਟ ਵੇਕਟਰ ਪੈਦਾ ਕਰਦੀ ਹੈ। ਇੱਕ ਅਰਥ ਵਿੱਚ, ਇਹ `Dense` ਲੇਅਰ ਦੇ ਬਹੁਤ ਹੀ ਸਮਾਨ ਹੈ, ਪਰ ਇਹ ਇੱਕ-ਹਾਟ ਐਨਕੋਡ ਕੀਤੇ ਵੇਕਟਰ ਨੂੰ ਇਨਪੁਟ ਵਜੋਂ ਲੈਣ ਦੀ ਬਜਾਏ, ਇੱਕ ਸ਼ਬਦ ਨੰਬਰ ਲੈ ਸਕਦੀ ਹੈ।

ਜਦੋਂ ਅਸੀਂ ਆਪਣੇ ਨੈਟਵਰਕ ਵਿੱਚ ਪਹਿਲੀ ਲੇਅਰ ਵਜੋਂ ਐਮਬੈਡਿੰਗ ਲੇਅਰ ਦੀ ਵਰਤੋਂ ਕਰਦੇ ਹਾਂ, ਤਾਂ ਅਸੀਂ ਬੈਗ-ਆਫ-ਵਰਡਸ ਮਾਡਲ ਤੋਂ **ਐਮਬੈਡਿੰਗ ਬੈਗ** ਮਾਡਲ ਵਿੱਚ ਸਵਿੱਚ ਕਰ ਸਕਦੇ ਹਾਂ, ਜਿੱਥੇ ਅਸੀਂ ਪਹਿਲਾਂ ਆਪਣੇ ਟੈਕਸਟ ਵਿੱਚ ਹਰ ਸ਼ਬਦ ਨੂੰ ਉਸ ਦੇ ਸੰਬੰਧਿਤ ਐਮਬੈਡਿੰਗ ਵਿੱਚ ਬਦਲਦੇ ਹਾਂ, ਅਤੇ ਫਿਰ ਉਹਨਾਂ ਸਾਰੀਆਂ ਐਮਬੈਡਿੰਗਜ਼ 'ਤੇ ਕੁਝ ਸਮੁੱਚੇ ਫੰਕਸ਼ਨ ਦੀ ਗਣਨਾ ਕਰਦੇ ਹਾਂ, ਜਿਵੇਂ ਕਿ `sum`, `average` ਜਾਂ `max`।  

![ਪੰਜ ਕ੍ਰਮਵਾਰ ਸ਼ਬਦਾਂ ਲਈ ਐਮਬੈਡਿੰਗ ਕਲਾਸੀਫਾਇਰ ਦਿਖਾਉਣ ਵਾਲੀ ਚਿੱਤਰ।](../../../../../translated_images/pa/embedding-classifier-example.b77f021a7ee67eee.webp)

ਸਾਡੇ ਕਲਾਸੀਫਾਇਰ ਨਿਊਰਲ ਨੈਟਵਰਕ ਵਿੱਚ ਹੇਠਾਂ ਦਿੱਤੀਆਂ ਲੇਅਰਾਂ ਸ਼ਾਮਲ ਹਨ:

* `TextVectorization` ਲੇਅਰ, ਜੋ ਇੱਕ ਸਟ੍ਰਿੰਗ ਨੂੰ ਇਨਪੁਟ ਵਜੋਂ ਲੈਂਦੀ ਹੈ ਅਤੇ ਟੋਕਨ ਨੰਬਰਾਂ ਦਾ ਟੈਂਸਰ ਪੈਦਾ ਕਰਦੀ ਹੈ। ਅਸੀਂ ਕੁਝ ਵਾਜਬ `vocab_size` ਨਿਰਧਾਰਤ ਕਰਾਂਗੇ ਅਤੇ ਘੱਟ ਵਰਤੇ ਜਾਣ ਵਾਲੇ ਸ਼ਬਦਾਂ ਨੂੰ ਅਣਡਿੱਠਾ ਕਰਾਂਗੇ। ਇਨਪੁਟ ਸ਼ੇਪ 1 ਹੋਵੇਗੀ, ਅਤੇ ਆਉਟਪੁੱਟ ਸ਼ੇਪ $n$ ਹੋਵੇਗੀ, ਕਿਉਂਕਿ ਅਸੀਂ ਨਤੀਜੇ ਵਜੋਂ $n$ ਟੋਕਨ ਪ੍ਰਾਪਤ ਕਰਾਂਗੇ, ਜਿਨ੍ਹਾਂ ਵਿੱਚੋਂ ਹਰ ਇੱਕ ਵਿੱਚ 0 ਤੋਂ `vocab_size` ਤੱਕ ਦੇ ਨੰਬਰ ਹੋਣਗੇ।
* `Embedding` ਲੇਅਰ, ਜੋ $n$ ਨੰਬਰਾਂ ਨੂੰ ਲੈਂਦੀ ਹੈ ਅਤੇ ਹਰ ਨੰਬਰ ਨੂੰ ਦਿੱਤੇ ਗਏ ਲੰਬਾਈ (ਸਾਡੇ ਉਦਾਹਰਨ ਵਿੱਚ 100) ਦੇ ਸੰਘਣੇ ਵੇਕਟਰ ਵਿੱਚ ਘਟਾਉਂਦੀ ਹੈ। ਇਸ ਤਰ੍ਹਾਂ, $n$ ਸ਼ੇਪ ਵਾਲੇ ਇਨਪੁਟ ਟੈਂਸਰ ਨੂੰ $n\times 100$ ਸ਼ੇਪ ਵਾਲੇ ਟੈਂਸਰ ਵਿੱਚ ਬਦਲਿਆ ਜਾਵੇਗਾ।
* ਐਗਰੀਗੇਸ਼ਨ ਲੇਅਰ, ਜੋ ਪਹਿਲੇ ਧੁਰੇ ਦੇ ਨਾਲ ਇਸ ਟੈਂਸਰ ਦਾ ਔਸਤ ਲੈਂਦੀ ਹੈ, ਅਰਥਾਤ ਇਹ ਵੱਖ-ਵੱਖ ਸ਼ਬਦਾਂ ਦੇ ਸਾਰੇ $n$ ਇਨਪੁਟ ਟੈਂਸਰਾਂ ਦਾ ਔਸਤ ਗਣਨਾ ਕਰੇਗੀ। ਇਸ ਲੇਅਰ ਨੂੰ ਲਾਗੂ ਕਰਨ ਲਈ, ਅਸੀਂ ਇੱਕ `Lambda` ਲੇਅਰ ਦੀ ਵਰਤੋਂ ਕਰਾਂਗੇ ਅਤੇ ਇਸ ਵਿੱਚ ਔਸਤ ਗਣਨਾ ਕਰਨ ਵਾਲਾ ਫੰਕਸ਼ਨ ਪਾਸ ਕਰਾਂਗੇ। ਆਉਟਪੁੱਟ ਦੀ ਸ਼ੇਪ 100 ਹੋਵੇਗੀ, ਅਤੇ ਇਹ ਪੂਰੇ ਇਨਪੁਟ ਕ੍ਰਮ ਦੀ ਗਿਣਤੀਗਤ ਪ੍ਰਸਤੁਤੀ ਹੋਵੇਗੀ।
* ਅੰਤਮ `Dense` ਲੀਨੀਅਰ ਕਲਾਸੀਫਾਇਰ।


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


`summary` ਪ੍ਰਿੰਟਆਉਟ ਵਿੱਚ, **output shape** ਕਾਲਮ ਵਿੱਚ, ਪਹਿਲਾ ਟੈਂਸਰ ਡਾਈਮੇਂਸ਼ਨ `None` ਮਿਨੀਬੈਚ ਸਾਈਜ਼ ਨੂੰ ਦਰਸਾਉਂਦਾ ਹੈ, ਅਤੇ ਦੂਜਾ ਟੋਕਨ ਕ੍ਰਮ ਦੀ ਲੰਬਾਈ ਨੂੰ ਦਰਸਾਉਂਦਾ ਹੈ। ਮਿਨੀਬੈਚ ਵਿੱਚ ਸਾਰੇ ਟੋਕਨ ਕ੍ਰਮਾਂ ਦੀਆਂ ਲੰਬਾਈਆਂ ਵੱਖ-ਵੱਖ ਹੁੰਦੀਆਂ ਹਨ। ਅਸੀਂ ਅਗਲੇ ਭਾਗ ਵਿੱਚ ਇਸ ਨਾਲ ਨਿਪਟਣ ਦੇ ਤਰੀਕੇ ਬਾਰੇ ਗੱਲ ਕਰਾਂਗੇ।

ਹੁਣ ਆਓ ਨੈੱਟਵਰਕ ਨੂੰ ਟ੍ਰੇਨ ਕਰੀਏ:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **ਨੋਟ** ਕਿ ਅਸੀਂ ਡਾਟਾ ਦੇ ਇੱਕ ਉਪਸੈੱਟ ਦੇ ਆਧਾਰ 'ਤੇ ਵੈਕਟੋਰਾਈਜ਼ਰ ਬਣਾ ਰਹੇ ਹਾਂ। ਇਹ ਪ੍ਰਕਿਰਿਆ ਨੂੰ ਤੇਜ਼ ਕਰਨ ਲਈ ਕੀਤਾ ਜਾਂਦਾ ਹੈ, ਅਤੇ ਇਸ ਨਾਲ ਇਹ ਸਥਿਤੀ ਬਣ ਸਕਦੀ ਹੈ ਕਿ ਸਾਡੇ ਟੈਕਸਟ ਦੇ ਸਾਰੇ ਟੋਕਨ ਸ਼ਬਦਾਵਲੀ ਵਿੱਚ ਮੌਜੂਦ ਨਾ ਹੋਣ। ਇਸ ਮਾਮਲੇ ਵਿੱਚ, ਉਹ ਟੋਕਨ ਨਜ਼ਰਅੰਦਾਜ਼ ਕਰ ਦਿੱਤੇ ਜਾਣਗੇ, ਜਿਸ ਨਾਲ ਥੋੜ੍ਹੀ ਘੱਟ ਸਹੀਤਾ ਹੋ ਸਕਦੀ ਹੈ। ਹਾਲਾਂਕਿ, ਅਸਲ ਜ਼ਿੰਦਗੀ ਵਿੱਚ ਟੈਕਸਟ ਦਾ ਇੱਕ ਉਪਸੈੱਟ ਅਕਸਰ ਇੱਕ ਚੰਗੀ ਸ਼ਬਦਾਵਲੀ ਅਨੁਮਾਨ ਦਿੰਦਾ ਹੈ।


### ਵੱਖ-ਵੱਖ ਲੰਬਾਈਆਂ ਵਾਲੀਆਂ ਵਰਿਆਬਲ ਲੜੀਆਂ ਨਾਲ ਨਿਪਟਣਾ

ਆਓ ਸਮਝੀਏ ਕਿ ਮਿਨੀਬੈਚਾਂ ਵਿੱਚ ਟ੍ਰੇਨਿੰਗ ਕਿਵੇਂ ਹੁੰਦੀ ਹੈ। ਉਪਰੋਕਤ ਉਦਾਹਰਨ ਵਿੱਚ, ਇਨਪੁਟ ਟੈਂਸਰ ਦਾ ਆਕਾਰ 1 ਹੈ, ਅਤੇ ਅਸੀਂ 128-ਲੰਬੀਆਂ ਮਿਨੀਬੈਚਾਂ ਦੀ ਵਰਤੋਂ ਕਰਦੇ ਹਾਂ, ਇਸ ਲਈ ਟੈਂਸਰ ਦਾ ਅਸਲ ਆਕਾਰ $128 \times 1$ ਹੈ। ਹਾਲਾਂਕਿ, ਹਰ ਵਾਕ ਵਿੱਚ ਟੋਕਨ ਦੀ ਗਿਣਤੀ ਵੱਖ-ਵੱਖ ਹੁੰਦੀ ਹੈ। ਜੇਕਰ ਅਸੀਂ `TextVectorization` ਲੇਅਰ ਨੂੰ ਇੱਕ ਇਨਪੁਟ 'ਤੇ ਲਾਗੂ ਕਰਦੇ ਹਾਂ, ਤਾਂ ਵਾਪਸ ਕੀਤੇ ਗਏ ਟੋਕਨ ਦੀ ਗਿਣਤੀ ਵੱਖ-ਵੱਖ ਹੁੰਦੀ ਹੈ, ਇਹ ਇਸ ਗੱਲ 'ਤੇ ਨਿਰਭਰ ਕਰਦਾ ਹੈ ਕਿ ਟੈਕਸਟ ਨੂੰ ਕਿਵੇਂ ਟੋਕਨਾਈਜ਼ ਕੀਤਾ ਗਿਆ ਹੈ:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


ਹਾਲਾਂਕਿ, ਜਦੋਂ ਅਸੀਂ ਕਈ ਕ੍ਰਮਾਂ 'ਤੇ ਵੈਕਟਰਾਈਜ਼ਰ ਲਾਗੂ ਕਰਦੇ ਹਾਂ, ਤਾਂ ਇਹ ਇੱਕ ਆਯਤਾਕਾਰ ਆਕਾਰ ਦਾ ਟੈਂਸਰ ਤਿਆਰ ਕਰਨਾ ਪੈਂਦਾ ਹੈ, ਇਸ ਲਈ ਇਹ ਅਣਵਰਤ ਤੱਤਾਂ ਨੂੰ PAD ਟੋਕਨ ਨਾਲ ਭਰਦਾ ਹੈ (ਜੋ ਸਾਡੇ ਮਾਮਲੇ ਵਿੱਚ ਜ਼ੀਰੋ ਹੈ):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

ਇੱਥੇ ਅਸੀਂ ਐਮਬੈਡਿੰਗਜ਼ ਦੇਖ ਸਕਦੇ ਹਾਂ:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **ਨੋਟ**: ਕੁਝ ਮਾਮਲਿਆਂ ਵਿੱਚ ਪੈਡਿੰਗ ਦੀ ਮਾਤਰਾ ਨੂੰ ਘਟਾਉਣ ਲਈ, ਸਾਰੇ ਕ੍ਰਮਾਂ ਨੂੰ ਡਾਟਾਸੈਟ ਵਿੱਚ ਲੰਬਾਈ ਵਧਣ ਦੇ ਕ੍ਰਮ ਵਿੱਚ ਸਾਰਟ ਕਰਨਾ ਵਾਜਬ ਹੈ (ਜਾਂ, ਜ਼ਿਆਦਾ ਸਹੀ ਤੌਰ 'ਤੇ, ਟੋਕਨ ਦੀ ਗਿਣਤੀ ਦੇ ਅਨੁਸਾਰ)। ਇਹ ਯਕੀਨੀ ਬਣਾਏਗਾ ਕਿ ਹਰ ਮਿਨੀਬੈਚ ਵਿੱਚ ਸਮਾਨ ਲੰਬਾਈ ਦੇ ਕ੍ਰਮ ਸ਼ਾਮਲ ਹਨ।


## ਸੈਮੈਂਟਿਕ ਐਮਬੈਡਿੰਗਸ: ਵਰਡ2ਵੈਕ

ਸਾਡੇ ਪਿਛਲੇ ਉਦਾਹਰਨ ਵਿੱਚ, ਐਮਬੈਡਿੰਗ ਲੇਅਰ ਨੇ ਸ਼ਬਦਾਂ ਨੂੰ ਵੈਕਟਰ ਪ੍ਰਤੀਨਿਧੀਆਂ ਵਿੱਚ ਮੈਪ ਕਰਨਾ ਸਿੱਖਿਆ, ਪਰ ਇਹ ਪ੍ਰਤੀਨਿਧੀਆਂ ਵਿੱਚ ਸੈਮੈਂਟਿਕ ਅਰਥ ਨਹੀਂ ਸੀ। ਇਹ ਵਧੀਆ ਹੋਵੇਗਾ ਕਿ ਅਸੀਂ ਇੱਕ ਵੈਕਟਰ ਪ੍ਰਤੀਨਿਧੀ ਸਿੱਖੀਏ ਜਿਸ ਵਿੱਚ ਸਮਾਨ ਸ਼ਬਦ ਜਾਂ ਪਰਿਆਯ ਸ਼ਬਦ ਕੁਝ ਵੈਕਟਰ ਦੂਰੀ (ਜਿਵੇਂ ਕਿ ਯੂਕਲਿਡੀਅਨ ਦੂਰੀ) ਦੇ ਹਿਸਾਬ ਨਾਲ ਇੱਕ ਦੂਜੇ ਦੇ ਨੇੜੇ ਹੋਣ।

ਇਸ ਨੂੰ ਕਰਨ ਲਈ, ਸਾਨੂੰ ਆਪਣੇ ਐਮਬੈਡਿੰਗ ਮਾਡਲ ਨੂੰ ਇੱਕ ਵੱਡੇ ਟੈਕਸਟ ਸੰਗ੍ਰਹਿ 'ਤੇ [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) ਵਰਗੇ ਤਕਨੀਕ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਪ੍ਰੀ-ਟ੍ਰੇਨ ਕਰਨ ਦੀ ਲੋੜ ਹੈ। ਇਹ ਦੋ ਮੁੱਖ ਆਰਕੀਟੈਕਚਰਾਂ 'ਤੇ ਆਧਾਰਿਤ ਹੈ ਜੋ ਸ਼ਬਦਾਂ ਦੀ ਵੰਡਵਾਂ ਪ੍ਰਤੀਨਿਧੀ ਪੈਦਾ ਕਰਨ ਲਈ ਵਰਤੇ ਜਾਂਦੇ ਹਨ:

 - **ਕੰਟਿਨਿਊਅਸ ਬੈਗ-ਆਫ-ਵਰਡਸ** (CBoW), ਜਿਸ ਵਿੱਚ ਅਸੀਂ ਮਾਡਲ ਨੂੰ ਆਸ-ਪਾਸ ਦੇ ਸੰਦਰਭ ਤੋਂ ਇੱਕ ਸ਼ਬਦ ਦੀ ਭਵਿੱਖਵਾਣੀ ਕਰਨ ਲਈ ਟ੍ਰੇਨ ਕਰਦੇ ਹਾਂ। ਦਿੱਤੇ ਗਏ ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$ ਵਿੱਚ, ਮਾਡਲ ਦਾ ਉਦੇਸ਼ $(W_{-2},W_{-1},W_1,W_2)$ ਤੋਂ $W_0$ ਦੀ ਭਵਿੱਖਵਾਣੀ ਕਰਨਾ ਹੈ।
 - **ਕੰਟਿਨਿਊਅਸ ਸਕਿਪ-ਗ੍ਰਾਮ** CBoW ਦੇ ਉਲਟ ਹੈ। ਮਾਡਲ ਸੰਦਰਭ ਸ਼ਬਦਾਂ ਦੀ ਆਸ-ਪਾਸ ਦੀ ਵਿੰਡੋ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਮੌਜੂਦਾ ਸ਼ਬਦ ਦੀ ਭਵਿੱਖਵਾਣੀ ਕਰਦਾ ਹੈ।

CBoW ਤੇਜ਼ ਹੈ, ਜਦਕਿ ਸਕਿਪ-ਗ੍ਰਾਮ ਹੌਲੀ ਹੈ, ਪਰ ਇਹ ਅਲਪ-ਵਰਤੋਂ ਵਾਲੇ ਸ਼ਬਦਾਂ ਦੀ ਵਧੀਆ ਪ੍ਰਤੀਨਿਧੀ ਕਰਦਾ ਹੈ।

![CBoW ਅਤੇ ਸਕਿਪ-ਗ੍ਰਾਮ ਐਲਗੋਰਿਥਮ ਦਿਖਾਉਣ ਵਾਲੀ ਚਿੱਤਰਕਾਰੀ ਜੋ ਸ਼ਬਦਾਂ ਨੂੰ ਵੈਕਟਰ ਵਿੱਚ ਬਦਲਦੀ ਹੈ।](../../../../../translated_images/pa/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Google News ਡੇਟਾਸੈਟ 'ਤੇ ਪ੍ਰੀ-ਟ੍ਰੇਨ ਕੀਤੇ ਵਰਡ2ਵੈਕ ਐਮਬੈਡਿੰਗ ਨਾਲ ਪ੍ਰਯੋਗ ਕਰਨ ਲਈ, ਅਸੀਂ **gensim** ਲਾਇਬ੍ਰੇਰੀ ਦੀ ਵਰਤੋਂ ਕਰ ਸਕਦੇ ਹਾਂ। ਹੇਠਾਂ ਅਸੀਂ 'neural' ਦੇ ਸਭ ਤੋਂ ਸਮਾਨ ਸ਼ਬਦਾਂ ਨੂੰ ਲੱਭਦੇ ਹਾਂ।

> **Note:** ਜਦੋਂ ਤੁਸੀਂ ਪਹਿਲੀ ਵਾਰ ਸ਼ਬਦ ਵੈਕਟਰ ਬਣਾਉਂਦੇ ਹੋ, ਤਾਂ ਇਹਨਾਂ ਨੂੰ ਡਾਊਨਲੋਡ ਕਰਨ ਵਿੱਚ ਕੁਝ ਸਮਾਂ ਲੱਗ ਸਕਦਾ ਹੈ!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


ਅਸੀਂ ਸ਼ਬਦ ਤੋਂ ਵੈਕਟਰ ਐਮਬੈਡਿੰਗ ਨੂੰ ਵੀ ਕੱਢ ਸਕਦੇ ਹਾਂ, ਜਿਸਨੂੰ ਵਰਗੀਕਰਨ ਮਾਡਲ ਦੀ ਟ੍ਰੇਨਿੰਗ ਵਿੱਚ ਵਰਤਿਆ ਜਾ ਸਕਦਾ ਹੈ। ਐਮਬੈਡਿੰਗ ਵਿੱਚ 300 ਘਟਕਾਂ ਹੁੰਦੀਆਂ ਹਨ, ਪਰ ਇੱਥੇ ਸਪਸ਼ਟਤਾ ਲਈ ਅਸੀਂ ਸਿਰਫ ਵੈਕਟਰ ਦੀਆਂ ਪਹਿਲੀਆਂ 20 ਘਟਕਾਂ ਦਿਖਾਉਂਦੇ ਹਾਂ:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

ਸਮਾਂਤਿਕ ਐਮਬੈਡਿੰਗਜ਼ ਬਾਰੇ ਵੱਡੀ ਗੱਲ ਇਹ ਹੈ ਕਿ ਤੁਸੀਂ ਸੈਮਾਂਟਿਕਸ ਦੇ ਆਧਾਰ 'ਤੇ ਵੈਕਟਰ ਕੋਡਿੰਗ ਨੂੰ ਮੋੜ ਸਕਦੇ ਹੋ। ਉਦਾਹਰਣ ਵਜੋਂ, ਅਸੀਂ ਉਹ ਸ਼ਬਦ ਲੱਭਣ ਲਈ ਕਹਿ ਸਕਦੇ ਹਾਂ ਜਿਸਦੀ ਵੈਕਟਰ ਪ੍ਰਤੀਨਿਧਤਾ ਸ਼ਬਦਾਂ *ਰਾਜਾ* ਅਤੇ *ਔਰਤ* ਦੇ ਸਭ ਤੋਂ ਨੇੜੇ ਹੋਵੇ, ਅਤੇ ਸ਼ਬਦ *ਮਰਦ* ਤੋਂ ਸਭ ਤੋਂ ਦੂਰ ਹੋਵੇ:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

ਉਪਰ ਦਿੱਤੇ ਉਦਾਹਰਨ ਵਿੱਚ ਕੁਝ ਅੰਦਰੂਨੀ GenSym ਜਾਦੂ ਦੀ ਵਰਤੋਂ ਕੀਤੀ ਗਈ ਹੈ, ਪਰ ਅਸਲ ਤਰਕ ਹਕੀਕਤ ਵਿੱਚ ਕਾਫੀ ਸਧਾਰਨ ਹੈ। ਐਮਬੈਡਿੰਗਜ਼ ਬਾਰੇ ਦਿਲਚਸਪ ਗੱਲ ਇਹ ਹੈ ਕਿ ਤੁਸੀਂ ਐਮਬੈਡਿੰਗ ਵੇਕਟਰਾਂ 'ਤੇ ਆਮ ਵੇਕਟਰ ਕਾਰਵਾਈਆਂ ਕਰ ਸਕਦੇ ਹੋ, ਅਤੇ ਇਹ ਸ਼ਬਦ ਦੇ **ਅਰਥਾਂ** 'ਤੇ ਕਾਰਵਾਈਆਂ ਨੂੰ ਦਰਸਾਏਗਾ। ਉਪਰ ਦਿੱਤਾ ਉਦਾਹਰਨ ਵੇਕਟਰ ਕਾਰਵਾਈਆਂ ਦੇ ਤੌਰ 'ਤੇ ਵਿਆਖਿਆ ਕੀਤਾ ਜਾ ਸਕਦਾ ਹੈ: ਅਸੀਂ **KING-MAN+WOMAN** ਦੇ ਅਨੁਕੂਲ ਵੇਕਟਰ ਦੀ ਗਣਨਾ ਕਰਦੇ ਹਾਂ (ਵੇਕਟਰ ਪ੍ਰਸਤੁਤੀਆਂ 'ਤੇ ਸੰਬੰਧਿਤ ਸ਼ਬਦਾਂ ਦੇ `+` ਅਤੇ `-` ਕਾਰਵਾਈਆਂ ਕੀਤੀਆਂ ਜਾਂਦੀਆਂ ਹਨ), ਅਤੇ ਫਿਰ ਉਸ ਵੇਕਟਰ ਦੇ ਸਭ ਤੋਂ ਨੇੜੇ ਸ਼ਬਦ ਨੂੰ ਸ਼ਬਦਕੋਸ਼ ਵਿੱਚ ਲੱਭਦੇ ਹਾਂ:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: ਸਾਨੂੰ *man* ਅਤੇ *woman* ਵੇਕਟਰਾਂ ਵਿੱਚ ਛੋਟੇ ਗੁਣਾਂ ਜੋੜਣੇ ਪਏ - ਇਸਨੂੰ ਹਟਾ ਕੇ ਦੇਖੋ ਕਿ ਕੀ ਹੁੰਦਾ ਹੈ।

ਸਭ ਤੋਂ ਨਜ਼ਦੀਕੀ ਵੇਕਟਰ ਲੱਭਣ ਲਈ, ਅਸੀਂ TensorFlow ਮਸ਼ੀਨਰੀ ਦੀ ਵਰਤੋਂ ਕਰਦੇ ਹਾਂ ਤਾਂ ਜੋ ਆਪਣੇ ਵੇਕਟਰ ਅਤੇ ਸ਼ਬਦਾਵਲੀ ਵਿੱਚ ਸਾਰੇ ਵੇਕਟਰਾਂ ਦੇ ਵਿਚਕਾਰ ਦੂਰੀਆਂ ਦਾ ਇੱਕ ਵੇਕਟਰ ਗਣਨਾ ਕਰ ਸਕੀਏ, ਅਤੇ ਫਿਰ `argmin` ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਘੱਟੋ-ਘੱਟ ਸ਼ਬਦ ਦਾ ਸੂਚਕ ਲੱਭਦੇ ਹਾਂ।


ਜਦੋਂ ਕਿ Word2Vec ਸ਼ਬਦ ਦੇ ਅਰਥ ਨੂੰ ਦਰਸਾਉਣ ਦਾ ਇੱਕ ਸ਼ਾਨਦਾਰ ਤਰੀਕਾ ਲੱਗਦਾ ਹੈ, ਇਸ ਵਿੱਚ ਕਈ ਘਾਟ ਹਨ, ਜਿਵੇਂ ਕਿ ਹੇਠਾਂ ਦਿੱਤੇ ਗਏ ਹਨ:

* ਦੋਵੇਂ CBoW ਅਤੇ skip-gram ਮਾਡਲ **ਪ੍ਰਡਿਕਟਿਵ ਐਮਬੈਡਿੰਗਸ** ਹਨ, ਅਤੇ ਇਹ ਸਿਰਫ਼ ਸਥਾਨਕ ਸੰਦਰਭ ਨੂੰ ਧਿਆਨ ਵਿੱਚ ਰੱਖਦੇ ਹਨ। Word2Vec ਗਲੋਬਲ ਸੰਦਰਭ ਦਾ ਫਾਇਦਾ ਨਹੀਂ ਲੈਂਦਾ।
* Word2Vec ਸ਼ਬਦ ਦੀ **ਮਾਰਫੋਲੋਜੀ** ਨੂੰ ਧਿਆਨ ਵਿੱਚ ਨਹੀਂ ਰੱਖਦਾ, ਜਿਵੇਂ ਕਿ ਇਹ ਹਕੀਕਤ ਕਿ ਸ਼ਬਦ ਦਾ ਅਰਥ ਸ਼ਬਦ ਦੇ ਵੱਖ-ਵੱਖ ਹਿੱਸਿਆਂ, ਜਿਵੇਂ ਕਿ ਜੜ, 'ਤੇ ਨਿਰਭਰ ਕਰ ਸਕਦਾ ਹੈ।

**FastText** ਦੂਜੇ ਪਾਬੰਦੀ ਨੂੰ ਪਾਰ ਕਰਨ ਦੀ ਕੋਸ਼ਿਸ਼ ਕਰਦਾ ਹੈ ਅਤੇ Word2Vec 'ਤੇ ਅਧਾਰਿਤ ਹੈ। ਇਹ ਹਰ ਸ਼ਬਦ ਅਤੇ ਉਸ ਵਿੱਚ ਪਾਏ ਜਾਣ ਵਾਲੇ ਅੱਖਰ n-grams ਲਈ ਵੈਕਟਰ ਪ੍ਰਤੀਨਿਧਤਾਵਾਂ ਸਿੱਖਦਾ ਹੈ। ਪ੍ਰਤੀਨਿਧਤਾਵਾਂ ਦੇ ਮੁੱਲਾਂ ਨੂੰ ਹਰ ਟ੍ਰੇਨਿੰਗ ਕਦਮ 'ਤੇ ਇੱਕ ਵੈਕਟਰ ਵਿੱਚ ਔਸਤ ਕੀਤਾ ਜਾਂਦਾ ਹੈ। ਜਦੋਂ ਕਿ ਇਹ ਪ੍ਰੀ-ਟ੍ਰੇਨਿੰਗ ਵਿੱਚ ਕਾਫ਼ੀ ਵਾਧੂ ਗਣਨਾ ਜੋੜਦਾ ਹੈ, ਇਹ ਸ਼ਬਦ ਐਮਬੈਡਿੰਗਸ ਨੂੰ ਸਬ-ਵਰਡ ਜਾਣਕਾਰੀ ਨੂੰ ਐਨਕੋਡ ਕਰਨ ਯੋਗ ਬਣਾਉਂਦਾ ਹੈ।

ਇੱਕ ਹੋਰ ਵਿਧੀ, **GloVe**, ਸ਼ਬਦ ਐਮਬੈਡਿੰਗਸ ਲਈ ਇੱਕ ਵੱਖਰਾ ਦ੍ਰਿਸ਼ਟੀਕੋਣ ਵਰਤਦੀ ਹੈ, ਜੋ ਕਿ ਸ਼ਬਦ-ਸੰਦਰਭ ਮੈਟ੍ਰਿਕਸ ਦੇ ਫੈਕਟਰਾਈਜ਼ੇਸ਼ਨ 'ਤੇ ਅਧਾਰਿਤ ਹੈ। ਪਹਿਲਾਂ, ਇਹ ਇੱਕ ਵੱਡਾ ਮੈਟ੍ਰਿਕਸ ਬਣਾਉਂਦੀ ਹੈ ਜੋ ਵੱਖ-ਵੱਖ ਸੰਦਰਭਾਂ ਵਿੱਚ ਸ਼ਬਦ ਦੇ ਆਵਿਰਤੀ ਗਿਣਤੀ ਨੂੰ ਦਰਸਾਉਂਦੀ ਹੈ, ਅਤੇ ਫਿਰ ਇਹ ਇਸ ਮੈਟ੍ਰਿਕਸ ਨੂੰ ਘੱਟ ਡਾਇਮੈਂਸ਼ਨ ਵਿੱਚ ਪ੍ਰਤੀਨਿਧਤ ਕਰਨ ਦੀ ਕੋਸ਼ਿਸ਼ ਕਰਦੀ ਹੈ ਜਿਸ ਵਿੱਚ ਰੀਕੰਸਟਰਕਸ਼ਨ ਲਾਸ ਘਟਾਇਆ ਜਾ ਸਕੇ।

gensim ਲਾਇਬ੍ਰੇਰੀ ਉਹ ਸ਼ਬਦ ਐਮਬੈਡਿੰਗਸ ਦਾ ਸਮਰਥਨ ਕਰਦੀ ਹੈ, ਅਤੇ ਤੁਸੀਂ ਉਨ੍ਹਾਂ ਨਾਲ ਪ੍ਰਯੋਗ ਕਰ ਸਕਦੇ ਹੋ ਜੇਕਰ ਤੁਸੀਂ ਉਪਰ ਦਿੱਤੇ ਮਾਡਲ ਲੋਡਿੰਗ ਕੋਡ ਨੂੰ ਬਦਲੋ।


## ਕੇਰਾਸ ਵਿੱਚ ਪਹਿਲਾਂ ਤੋਂ ਤਿਆਰ ਕੀਤੇ ਐਮਬੈਡਿੰਗਜ਼ ਦੀ ਵਰਤੋਂ

ਅਸੀਂ ਉਪਰੋਕਤ ਉਦਾਹਰਨ ਨੂੰ ਇਸ ਤਰ੍ਹਾਂ ਸੋਧ ਸਕਦੇ ਹਾਂ ਕਿ ਸਾਡੇ ਐਮਬੈਡਿੰਗ ਲੇਅਰ ਵਿੱਚ ਮੈਟ੍ਰਿਕਸ ਨੂੰ ਸੈਮਾਂਟਿਕ ਐਮਬੈਡਿੰਗਜ਼, ਜਿਵੇਂ ਕਿ Word2Vec ਨਾਲ ਪਹਿਲਾਂ ਹੀ ਭਰਿਆ ਜਾਵੇ। ਪਹਿਲਾਂ ਤੋਂ ਤਿਆਰ ਕੀਤੇ ਐਮਬੈਡਿੰਗ ਅਤੇ ਟੈਕਸਟ ਕੋਰਪਸ ਦੇ ਸ਼ਬਦਾਵਲੀ ਅਕਸਰ ਮਿਲਦੇ ਨਹੀਂ ਹਨ, ਇਸ ਲਈ ਸਾਨੂੰ ਇੱਕ ਚੋਣ ਕਰਨੀ ਪਵੇਗੀ। ਇੱਥੇ ਅਸੀਂ ਦੋ ਸੰਭਾਵਿਤ ਵਿਕਲਪਾਂ ਦੀ ਜਾਂਚ ਕਰਦੇ ਹਾਂ: ਟੋਕਨਾਈਜ਼ਰ ਸ਼ਬਦਾਵਲੀ ਦੀ ਵਰਤੋਂ ਕਰਨਾ, ਅਤੇ Word2Vec ਐਮਬੈਡਿੰਗਜ਼ ਤੋਂ ਸ਼ਬਦਾਵਲੀ ਦੀ ਵਰਤੋਂ ਕਰਨਾ।

### ਟੋਕਨਾਈਜ਼ਰ ਸ਼ਬਦਾਵਲੀ ਦੀ ਵਰਤੋਂ

ਜਦੋਂ ਟੋਕਨਾਈਜ਼ਰ ਸ਼ਬਦਾਵਲੀ ਦੀ ਵਰਤੋਂ ਕੀਤੀ ਜਾਂਦੀ ਹੈ, ਤਾਂ ਸ਼ਬਦਾਵਲੀ ਦੇ ਕੁਝ ਸ਼ਬਦਾਂ ਲਈ Word2Vec ਐਮਬੈਡਿੰਗਜ਼ ਹੋਣਗੇ, ਅਤੇ ਕੁਝ ਗੁੰਮ ਹੋਣਗੇ। ਜਿਵੇਂ ਕਿ ਸਾਡੀ ਸ਼ਬਦਾਵਲੀ ਦਾ ਆਕਾਰ `vocab_size` ਹੈ, ਅਤੇ Word2Vec ਐਮਬੈਡਿੰਗ ਵੇਕਟਰ ਦੀ ਲੰਬਾਈ `embed_size` ਹੈ, ਐਮਬੈਡਿੰਗ ਲੇਅਰ ਨੂੰ `vocab_size`$\times$`embed_size` ਦੇ ਆਕਾਰ ਦੇ ਵਜ਼ਨ ਮੈਟ੍ਰਿਕਸ ਦੁਆਰਾ ਦਰਸਾਇਆ ਜਾਵੇਗਾ। ਅਸੀਂ ਇਸ ਮੈਟ੍ਰਿਕਸ ਨੂੰ ਸ਼ਬਦਾਵਲੀ ਵਿੱਚੋਂ ਗੁਜ਼ਰ ਕੇ ਭਰਾਂਗੇ:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


ਸ਼ਬਦਾਂ ਲਈ ਜੋ Word2Vec ਸ਼ਬਦਕੋਸ਼ ਵਿੱਚ ਮੌਜੂਦ ਨਹੀਂ ਹਨ, ਅਸੀਂ ਉਨ੍ਹਾਂ ਨੂੰ ਜ਼ੀਰੋਜ਼ ਵਜੋਂ ਛੱਡ ਸਕਦੇ ਹਾਂ, ਜਾਂ ਇੱਕ ਰੈਂਡਮ ਵੈਕਟਰ ਤਿਆਰ ਕਰ ਸਕਦੇ ਹਾਂ।

ਹੁਣ ਅਸੀਂ ਪ੍ਰੀ-ਟ੍ਰੇਨਡ ਵਜ਼ਨਾਂ ਨਾਲ ਇੱਕ ਐਮਬੈਡਿੰਗ ਲੇਅਰ ਨੂੰ ਪਰਿਭਾਸ਼ਿਤ ਕਰ ਸਕਦੇ ਹਾਂ:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **ਨੋਟ**: ਧਿਆਨ ਦਿਓ ਕਿ ਜਦੋਂ ਅਸੀਂ `Embedding` ਬਣਾਉਂਦੇ ਸਮੇਂ `trainable=False` ਸੈਟ ਕਰਦੇ ਹਾਂ, ਤਾਂ ਇਸਦਾ ਮਤਲਬ ਹੈ ਕਿ ਅਸੀਂ Embedding ਲੇਅਰ ਨੂੰ ਮੁੜ-ਟ੍ਰੇਨ ਨਹੀਂ ਕਰ ਰਹੇ। ਇਸ ਕਾਰਨ ਸਹੀਤਾ ਥੋੜੀ ਘੱਟ ਹੋ ਸਕਦੀ ਹੈ, ਪਰ ਇਹ ਟ੍ਰੇਨਿੰਗ ਦੀ ਗਤੀ ਨੂੰ ਤੇਜ਼ ਕਰਦਾ ਹੈ।

### ਐਮਬੈਡਿੰਗ ਵੋਕੈਬੁਲਰੀ ਦੀ ਵਰਤੋਂ

ਪਿਛਲੇ ਤਰੀਕੇ ਨਾਲ ਇੱਕ ਸਮੱਸਿਆ ਇਹ ਹੈ ਕਿ TextVectorization ਅਤੇ Embedding ਵਿੱਚ ਵਰਤੇ ਗਏ ਵੋਕੈਬੁਲਰੀਜ਼ ਵੱਖਰੇ ਹਨ। ਇਸ ਸਮੱਸਿਆ ਨੂੰ ਹੱਲ ਕਰਨ ਲਈ, ਅਸੀਂ ਹੇਠਾਂ ਦਿੱਤੇ ਤਰੀਕਿਆਂ ਵਿੱਚੋਂ ਕੋਈ ਇੱਕ ਵਰਤ ਸਕਦੇ ਹਾਂ:
* ਆਪਣੇ ਵੋਕੈਬੁਲਰੀ 'ਤੇ Word2Vec ਮਾਡਲ ਨੂੰ ਮੁੜ-ਟ੍ਰੇਨ ਕਰੋ।
* ਪ੍ਰੀ-ਟ੍ਰੇਨ ਕੀਤੇ Word2Vec ਮਾਡਲ ਦੇ ਵੋਕੈਬੁਲਰੀ ਨਾਲ ਸਾਡੇ ਡਾਟਾਸੈਟ ਨੂੰ ਲੋਡ ਕਰੋ। ਡਾਟਾਸੈਟ ਨੂੰ ਲੋਡ ਕਰਨ ਲਈ ਵਰਤੇ ਜਾਣ ਵਾਲੇ ਵੋਕੈਬੁਲਰੀ ਨੂੰ ਲੋਡਿੰਗ ਦੌਰਾਨ ਨਿਰਧਾਰਤ ਕੀਤਾ ਜਾ ਸਕਦਾ ਹੈ।

ਦੂਜਾ ਤਰੀਕਾ ਆਸਾਨ ਲੱਗਦਾ ਹੈ, ਇਸ ਲਈ ਆਓ ਇਸਨੂੰ ਲਾਗੂ ਕਰੀਏ। ਸਭ ਤੋਂ ਪਹਿਲਾਂ, ਅਸੀਂ Word2Vec embeddings ਤੋਂ ਲਏ ਗਏ ਨਿਰਧਾਰਤ ਵੋਕੈਬੁਲਰੀ ਨਾਲ ਇੱਕ `TextVectorization` ਲੇਅਰ ਬਣਾਵਾਂਗੇ:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

ਜੈਨਸਿਮ ਵਰਡ ਐਮਬੈਡਿੰਗਸ ਲਾਇਬ੍ਰੇਰੀ ਵਿੱਚ ਇੱਕ ਸੁਵਿਧਾਜਨਕ ਫੰਕਸ਼ਨ, `get_keras_embeddings`, ਸ਼ਾਮਲ ਹੈ, ਜੋ ਤੁਹਾਡੇ ਲਈ ਆਪਣੇ ਆਪ ਹੀ ਸੰਬੰਧਿਤ ਕੇਰਾਸ ਐਮਬੈਡਿੰਗਸ ਲੇਅਰ ਬਣਾਏਗਾ।


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


ਸਾਡੇ ਡੇਟਾਸੈਟ ਦੇ ਕੁਝ ਸ਼ਬਦ ਪ੍ਰੀ-ਟ੍ਰੇਨਡ GloVe ਸ਼ਬਦਾਵਲੀ ਵਿੱਚ ਗੁੰਮ ਹਨ, ਜਿਸ ਕਰਕੇ ਉਹ ਮੁਢਲੀ ਤੌਰ 'ਤੇ ਅਣਗੌਰ ਕੀਤੇ ਜਾਂਦੇ ਹਨ। ਇਸਨੂੰ ਦੂਰ ਕਰਨ ਲਈ, ਅਸੀਂ ਆਪਣੇ ਡੇਟਾਸੈਟ ਦੇ ਆਧਾਰ 'ਤੇ ਆਪਣੇ ਐਮਬੈਡਿੰਗਜ਼ ਨੂੰ ਟ੍ਰੇਨ ਕਰ ਸਕਦੇ ਹਾਂ।


## ਸੰਦਰਭਿਕ ਐਮਬੈਡਿੰਗਸ

ਪ੍ਰੰਪਰਾਗਤ ਪ੍ਰੀ-ਟ੍ਰੇਨ ਕੀਤੇ ਐਮਬੈਡਿੰਗ ਪ੍ਰਤੀਨਿਧੀਆਂ ਜਿਵੇਂ ਕਿ Word2Vec ਦੀ ਇੱਕ ਮੁੱਖ ਸੀਮਿਤਤਾ ਇਹ ਹੈ ਕਿ, ਹਾਲਾਂਕਿ ਇਹ ਸ਼ਬਦ ਦੇ ਕੁਝ ਅਰਥ ਨੂੰ ਕੈਪਚਰ ਕਰ ਸਕਦੇ ਹਨ, ਪਰ ਇਹ ਵੱਖ-ਵੱਖ ਅਰਥਾਂ ਵਿੱਚ ਫਰਕ ਨਹੀਂ ਕਰ ਸਕਦੇ। ਇਹ ਹੇਠਾਂ ਦਿੱਤੇ ਮਾਡਲਾਂ ਵਿੱਚ ਸਮੱਸਿਆ ਪੈਦਾ ਕਰ ਸਕਦਾ ਹੈ।

ਉਦਾਹਰਨ ਵਜੋਂ, 'play' ਸ਼ਬਦ ਦੇ ਵੱਖ-ਵੱਖ ਅਰਥ ਹਨ ਜਿਵੇਂ ਕਿ ਇਹ ਦੋ ਵਾਕਾਂ ਵਿੱਚ:
- ਮੈਂ ਥੀਏਟਰ ਵਿੱਚ ਇੱਕ **play** ਦੇਖਣ ਗਿਆ।
- ਜੌਨ ਆਪਣੇ ਦੋਸਤਾਂ ਨਾਲ **play** ਕਰਨਾ ਚਾਹੁੰਦਾ ਹੈ।

ਜਿਨ ਪ੍ਰੀ-ਟ੍ਰੇਨ ਕੀਤੇ ਐਮਬੈਡਿੰਗਸ ਦੀ ਗੱਲ ਕੀਤੀ ਗਈ ਹੈ, ਉਹ 'play' ਸ਼ਬਦ ਦੇ ਦੋਨੋ ਅਰਥਾਂ ਨੂੰ ਇੱਕੋ ਜਿਹੇ ਐਮਬੈਡਿੰਗ ਵਿੱਚ ਪ੍ਰਤੀਨਿਧਿਤ ਕਰਦੇ ਹਨ। ਇਸ ਸੀਮਿਤਤਾ ਨੂੰ ਦੂਰ ਕਰਨ ਲਈ, ਸਾਨੂੰ **ਭਾਸ਼ਾ ਮਾਡਲ** ਦੇ ਆਧਾਰ 'ਤੇ ਐਮਬੈਡਿੰਗ ਬਣਾਉਣ ਦੀ ਲੋੜ ਹੈ, ਜੋ ਕਿ ਵੱਡੇ ਟੈਕਸਟ ਕੋਰਪਸ 'ਤੇ ਟ੍ਰੇਨ ਕੀਤਾ ਜਾਂਦਾ ਹੈ ਅਤੇ *ਜਾਣਦਾ ਹੈ* ਕਿ ਸ਼ਬਦਾਂ ਨੂੰ ਵੱਖ-ਵੱਖ ਸੰਦਰਭਾਂ ਵਿੱਚ ਕਿਵੇਂ ਜੋੜਿਆ ਜਾ ਸਕਦਾ ਹੈ। ਸੰਦਰਭਿਕ ਐਮਬੈਡਿੰਗਸ ਬਾਰੇ ਚਰਚਾ ਇਸ ਟਿਊਟੋਰਿਅਲ ਦੇ ਦਾਇਰੇ ਤੋਂ ਬਾਹਰ ਹੈ, ਪਰ ਅਸੀਂ ਇਸ ਬਾਰੇ ਅਗਲੇ ਯੂਨਿਟ ਵਿੱਚ ਭਾਸ਼ਾ ਮਾਡਲਾਂ ਦੀ ਗੱਲ ਕਰਦੇ ਸਮੇਂ ਵਾਪਸ ਆਵਾਂਗੇ।



---

**ਅਸਵੀਕਰਤਾ**:  
ਇਹ ਦਸਤਾਵੇਜ਼ AI ਅਨੁਵਾਦ ਸੇਵਾ [Co-op Translator](https://github.com/Azure/co-op-translator) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਨੁਵਾਦ ਕੀਤਾ ਗਿਆ ਹੈ। ਜਦੋਂ ਕਿ ਅਸੀਂ ਸਹੀ ਹੋਣ ਦੀ ਕੋਸ਼ਿਸ਼ ਕਰਦੇ ਹਾਂ, ਕਿਰਪਾ ਕਰਕੇ ਧਿਆਨ ਦਿਓ ਕਿ ਸਵੈਚਾਲਿਤ ਅਨੁਵਾਦਾਂ ਵਿੱਚ ਗਲਤੀਆਂ ਜਾਂ ਅਸੁਚੀਤਤਾਵਾਂ ਹੋ ਸਕਦੀਆਂ ਹਨ। ਇਸ ਦੀ ਮੂਲ ਭਾਸ਼ਾ ਵਿੱਚ ਮੌਜੂਦ ਮੂਲ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਅਧਿਕਾਰਤ ਸਰੋਤ ਮੰਨਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ। ਮਹੱਤਵਪੂਰਨ ਜਾਣਕਾਰੀ ਲਈ, ਪੇਸ਼ੇਵਰ ਮਨੁੱਖੀ ਅਨੁਵਾਦ ਦੀ ਸਿਫਾਰਸ਼ ਕੀਤੀ ਜਾਂਦੀ ਹੈ। ਇਸ ਅਨੁਵਾਦ ਦੀ ਵਰਤੋਂ ਤੋਂ ਪੈਦਾ ਹੋਣ ਵਾਲੇ ਕਿਸੇ ਵੀ ਗਲਤਫਹਿਮੀ ਜਾਂ ਗਲਤ ਵਿਆਖਿਆ ਲਈ ਅਸੀਂ ਜ਼ਿੰਮੇਵਾਰ ਨਹੀਂ ਹਾਂ।
